In [47]:
import cudf as pd
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(data=boston.data, columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
        'TAX', 'PTRATIO', 'B', 'LSTAT'])
df['target'] = boston.target
df.head(5)

In [48]:
import cupy as np
X=df.iloc[:,0:13]
Y=df.iloc[:,13]
Y=np.array(Y)
Y=Y.reshape(-1,1)

In [49]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X.as_matrix())

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)

In [51]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [52]:
from cuml import LinearRegression

In [53]:
# import pandas as pd
df2 = pd.DataFrame()
mse = []
r2 = []
mae = []

In [54]:
algo = ["svd", "eig", "qr", "svd-qr", "svd-jacobi"]
for i in algo:
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(X_train,y_train)
    preds = lr.predict(X_test)
    
    mse.append(cuml.metrics.regression.mean_squared_error(y_test,preds))
    r2.append(cuml.metrics.regression.r2_score(y_test,preds))
    mae.append(cuml.metrics.regression.mean_absolute_error(y_test,preds))

In [55]:
df2['algo'] = algo
df2['mse'] = mse
df2['r2'] = r2
df2['mae'] = mae

In [56]:
df2

In [57]:
# print("Coefficients:")
# print(reg.coef_)
# print("Intercept:")
# print(reg.intercept_)